In [1]:
# ------------------------------------------------
# 라이브러리 import
# ------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json


### 데이터 불러오기

In [ ]:
file_path = 'data/LOCAL_PEOPLE_DONG/preprocessed_LOCAL_PEOPLE_DONG_all.csv'


,기준일ID,월,일,요일,시간대구분,행정동코드,시도명,시군구명,행정동명,총,...,여자25세부터29세,여자30세부터34세,여자35세부터39세,여자40세부터44세,여자45세부터49세,여자50세부터54세,여자55세부터59세,여자60세부터64세,여자65세부터69세,여자70세이상
0,2024-04-01,4,1,Mon,0,1144058500,서울,마포구,도화동,16690.3005,...,750.5231,790.1085,839.8785,697.5542,790.7150,636.1098,619.3970,593.8115,441.4222,1172.2903
1,2024-04-01,4,1,Mon,0,1129064000,서울,성북구,정릉3동,17254.8326,...,732.8108,488.7919,414.1525,376.4850,365.5632,455.2348,540.8218,412.3597,282.3098,1102.3257
2,2024-04-01,4,1,Mon,0,1126059000,서울,중랑구,상봉2동,24836.1736,...,2043.5564,1615.6401,1424.6506,831.8179,784.6880,769.7451,859.3055,761.8320,514.3764,1342.1258
3,2024-04-01,4,1,Mon,0,1153054000,서울,구로구,구로3동,26975.9704,...,1510.6376,1321.0909,1352.3565,1060.4784,1042.7210,782.7908,823.7957,799.9066,553.7243,1267.0419
4,2024-04-01,4,1,Mon,0,1150054000,서울,강서구,화곡1동,49512.7524,...,3170.3796,2592.4259,2458.9594,1956.3342,1870.8576,1692.5864,1928.3318,1546.4568,1473.8741,3217.3606


### 조건 설정

In [38]:

# 날짜
month = 12
day = 31

# 요일 
daynames = None

# 시간대 
times = [14]

# 나이 
ages = ['여자70세이상']
target_cols = ['기준일ID', '요일','시간대구분','행정동코드','시군구명','행정동명']+ages



In [39]:
df.columns

Index(['기준일ID', '월', '일', '요일', '시간대구분', '행정동코드', '시도명', '시군구명', '행정동명', '총',
       '남자0세부터9세', '남자10세부터14세', '남자15세부터19세', '남자20세부터24세', '남자25세부터29세',
       '남자30세부터34세', '남자35세부터39세', '남자40세부터44세', '남자45세부터49세', '남자50세부터54세',
       '남자55세부터59세', '남자60세부터64세', '남자65세부터69세', '남자70세이상', '여자0세부터9세',
       '여자10세부터14세', '여자15세부터19세', '여자20세부터24세', '여자25세부터29세', '여자30세부터34세',
       '여자35세부터39세', '여자40세부터44세', '여자45세부터49세', '여자50세부터54세', '여자55세부터59세',
       '여자60세부터64세', '여자65세부터69세', '여자70세이상'],
      dtype='object')

### 조건에 맞는 데이터 추출

,기준일ID,요일,시간대구분,행정동코드,시군구명,행정동명,여자70세이상
2794160,2024-12-31,Tue,14,1111054000,종로구,삼청동,433.4550
2794161,2024-12-31,Tue,14,1120062000,성동구,금호4가동,866.3651
2794162,2024-12-31,Tue,14,1159060500,동작구,흑석동,1864.2541
2794163,2024-12-31,Tue,14,1117051000,용산구,후암동,772.8355
2794164,2024-12-31,Tue,14,1171071000,송파구,잠실6동,2843.4995
...,...,...,...,...,...,...,...
2794579,2024-12-31,Tue,14,1114054000,중구,회현동,3763.1430
2794580,2024-12-31,Tue,14,1144056500,마포구,공덕동,1864.6969
2794581,2024-12-31,Tue,14,1150061500,강서구,우장산동,2424.8641
2794582,2024-12-31,Tue,14,1153055000,구로구,구로4동,1350.0120


### 합계 컬럼 생성

,기준일ID,요일,시간대구분,행정동코드,시군구명,행정동명,여자70세이상,합계
2794160,2024-12-31,Tue,14,1111054000,종로구,삼청동,433.4550,433.4550
2794161,2024-12-31,Tue,14,1120062000,성동구,금호4가동,866.3651,866.3651
2794162,2024-12-31,Tue,14,1159060500,동작구,흑석동,1864.2541,1864.2541
2794163,2024-12-31,Tue,14,1117051000,용산구,후암동,772.8355,772.8355
2794164,2024-12-31,Tue,14,1171071000,송파구,잠실6동,2843.4995,2843.4995


### 시각화
* geojson 다운로드
    * https://github.com/vuski/admdongkor
    * **생활인구 데이터는 데이터 일관성 유지를 위해 2016년 집계구, 행정동 코드를 사용하고 있음**

In [ ]:
############################################################################################
# 1. 서울시청을 중심으로 지도 생성
############################################################################################
# 서울시 위도, 경도
loc = (37.5642135,127.0016985)

m = folium.Map(location=loc
            , zoom_start=11
            # , tiles = 'CartoDB Positron'
            , width=600
            , height=400
            )

############################################################################################
# 2. geojson 데이터를 dict로 파싱 후 서울시만 필터링
############################################################################################
with open('geojson/HangJeongDong_ver20161231.geojson', mode='r', encoding='utf-8') as f:
    geo_dong = json.loads(f.read())

# 서울시 데이터만 필터링
# 행정동코드의 앞 2자리가 시군구를 나타냄
seoul_geo = {
    "type": "FeatureCollection",
    "features": [feature for feature in geo_dong["features"]
                if feature["properties"]["adm_cd"][:2] == "11"]
}

# ############################################################################################
# # 3. choropleth 시각화
# ############################################################################################



### 시각화 결과


날짜:12월 31일
요일:None
시간대:[14]
나이:['여자70세이상']

